# Creating CREATE File from Data

In [1]:
pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.9.0-py3-none-any.whl size=259467 sha256=2b6a8d8aee2b2f21069d622adf7861f0becddabe2ac76c1309a5946034de4fd9
  Stored in directory: /home/jovyan/.cache/pip/wheels/5a/6a/b3/89ab0efac04cf204cfc65f2bff4776865cfe5ff2e1b94ebaff
Successfully built neo4j
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import random
import csv

## Queries

In [11]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [12]:
conn = Neo4jConnection(uri="bolt://graph_db:7687", user="neo4j", pwd="password")

In [13]:
follows_file = open('twitter_combined.txt', 'r')
lines = follows_file.readlines()

follows = []
users = set()
for line in lines:
    line = line.replace('\n','')
    id1, id2 = line.split(' ')
    users.add(id1)
    users.add(id2)
    follows.append([id1, id2])

In [14]:
tweets = pd.read_csv("tweets.csv")
f = open("import/tweetsDB.csv", "w")
writer = csv.writer(f)
writer.writerow(["tweetId","content","date_time","language",":Label"])
count = 1
for index, tweet in tweets.iterrows():
    writer.writerow([count,tweet["content"],tweet["date_time"],tweet["language"],"TWEET"])
    count += 1
f.close()

In [15]:
count = 0
f = open("import/users.csv", "w")
f.write("userId,name,:Label\n")
for user in users:
   f.write(user + ",user" + str(count) + ",USER\n")
   count += 1
f.close()

In [16]:
f = open("import/follows.csv", "w")
f.write("userId,followerId,:TYPE\n")
for follow in follows:
    f.write(follow[0] + "," + follow[1] + ",FOLLOWS\n")
f.close()

In [17]:
conn.query(f"MATCH (n) RETURN n LIMIT 1;")

[<Record n=<Node element_id='4:e1a2f8d3-a31f-446e-9e94-2a84f08f0ca3:0' labels=frozenset({'User'}) properties={'name': 'user0', 'userId': '16404379'}>>]

In [10]:
conn.query("LOAD CSV WITH HEADERS FROM 'file:///users.csv' AS row CREATE (:User {userId: row.userId, name: row.name})")
conn.query("LOAD CSV WITH HEADERS FROM 'file:///tweetsDB.csv' AS row CREATE (:Tweet {tweetId: row.tweetId, content: row.content, date_time: row.date_time, language: row.language})")

Query failed: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(61845) already exists with label `User` and property `userId` = '9003112' (Failure when processing file '/var/lib/neo4j/import/users.csv' on line 2.)}
Query failed: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(81306) already exists with label `Tweet` and property `tweetId` = '1' (Failure when processing file '/var/lib/neo4j/import/tweetsDB.csv' on line 2.)}


In [11]:
conn.query("CREATE CONSTRAINT user_constraint IF NOT EXISTS FOR (u:User) REQUIRE u.userId IS UNIQUE")
conn.query("CREATE CONSTRAINT tweet_constraint IF NOT EXISTS FOR (t:Tweet) REQUIRE t.tweetId IS UNIQUE")

[]

In [12]:
conn.query("LOAD CSV WITH HEADERS FROM 'file:///follows.csv' AS row MATCH (e:User {userId: row.userId}) MATCH (c:User {userId: row.followerId}) MERGE (e)-[:FOLLOWS]->(c)")

[]

In [13]:
top100 = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN b, COUNT(a) as followers ORDER BY followers DESC LIMIT 100")
user_ids = []
for record in top100:
    user_ids.append(record.get("b").get("userId"))

tweets = pd.read_csv("import/tweetsDB.csv")
f = open("import/posted.csv", "w")
f.write("userId,tweetId,:TYPE\n")
for index, tweet in tweets.iterrows():
    f.write(f'{random.choice(user_ids)},{tweet["tweetId"]},POSTED\n')
f.close()

In [14]:
conn.query("LOAD CSV WITH HEADERS FROM 'file:///posted.csv' AS row MATCH (u:User {userId: row.userId}) MATCH (t:Tweet {tweetId: row.tweetId}) MERGE (u)-[:POSTED]->(t)")

[]

In [ ]:
top100 = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN b, COUNT(a) as followers ORDER BY followers DESC LIMIT 100")
allTweets = conn.query("Match (t: Tweet) RETURN t");
f = open("import/likes.csv","w")
f.write("userId,tweetId,:TYPE\n")

for user in top100: 
    followers = conn.query(f"MATCH (follower:User)-[:FOLLOWS]->(following:User) WHERE following.userId ='{user['b']['userId']}' RETURN follower")
    tweets = conn.query(f"MATCH (u:User {{userId: '{user['b']['userId']}'}})-[:POSTED]->(t:Tweet) RETURN t");
    for tweet in tweets: 
        for follower in followers:
            shouldLike = random.random() < 0.1
            if shouldLike:
                f.write(f"{follower['follower']['userId']},{tweet['t']['tweetId']},LIKES\n")

f.close()    

In [ ]:
%%time

conn.query("LOAD CSV WITH HEADERS FROM 'file:///likes.csv' as row CALL { with row MATCH (u:User {userId: row.userId}) MATCH (t:Tweet {tweetId: row.tweetId}) MERGE (u)-[:LIKES]->(t) }  IN TRANSACTIONS OF 100000 ROWS")

# Top 100 User mit den meisten Followern

In [19]:
top100 = conn.query("MATCH (a)-[:FOLLOWS]->(b) RETURN b, COUNT(a) as followers ORDER BY followers DESC LIMIT 100")

for user in top100:
    print(user['b']['name'], "hat", user['followers'])

user23516 :  3383
user25074 :  3216
user1781 :  2735
user50940 :  2647
user69469 :  2471
user40743 :  2462
user33433 :  2133
user38752 :  2074
user14303 :  1905
user29918 :  1707
user73980 :  1632
user31252 :  1591
user56892 :  1521
user56971 :  1503
user66477 :  1500
user77339 :  1410
user77394 :  1402
user4665 :  1370
user21904 :  1337
user58114 :  1315
user47959 :  1273
user45639 :  1255
user71012 :  1231
user19416 :  1214
user1733 :  1201
user18127 :  1198
user58309 :  1186
user52387 :  1167
user63035 :  1154
user27752 :  1123
user77284 :  1120
user64715 :  1108
user33440 :  1098
user77008 :  1088
user76017 :  1072
user80101 :  1070
user10026 :  1059
user19035 :  1053
user60306 :  1047
user11550 :  1036
user41077 :  1022
user65544 :  1016
user23821 :  1014
user31073 :  992
user54692 :  989
user35681 :  977
user40788 :  957
user74879 :  928
user25302 :  920
user115 :  915
user27502 :  915
user77805 :  913
user24501 :  911
user47582 :  900
user38931 :  894
user66649 :  889
user34800 

# Top 100 User, die den meisten Userns der Top 100 folgen

In [31]:
query = """MATCH (follower:User)-[follows:FOLLOWS]->(followed:User)
WHERE followed.userId IN [""" 

count = 0
for user in top100:
    query += f"'{user['b']['userId']}'"
    if count < 99: query += ","
    count += 1

query += """]
WITH follower, COUNT(follows) AS numFollowed
ORDER BY numFollowed DESC
RETURN follower.name AS username, numFollowed
LIMIT 100
"""

result = conn.query(query)

In [33]:
for record in result:
    print(record['username'], "folgt", record['numFollowed'], 'Usern')

user14303 folgt 69 Usern
user41327 folgt 55 Usern
user2169 folgt 50 Usern
user41320 folgt 49 Usern
user33440 folgt 47 Usern
user64029 folgt 46 Usern
user29265 folgt 44 Usern
user38908 folgt 41 Usern
user29314 folgt 40 Usern
user36591 folgt 39 Usern
user12499 folgt 38 Usern
user33065 folgt 38 Usern
user74270 folgt 37 Usern
user21544 folgt 37 Usern
user51566 folgt 37 Usern
user29427 folgt 36 Usern
user67590 folgt 35 Usern
user71341 folgt 35 Usern
user66389 folgt 35 Usern
user41077 folgt 35 Usern
user61428 folgt 34 Usern
user38931 folgt 34 Usern
user46348 folgt 34 Usern
user64184 folgt 34 Usern
user12403 folgt 34 Usern
user37319 folgt 34 Usern
user76560 folgt 33 Usern
user31252 folgt 33 Usern
user3856 folgt 33 Usern
user68699 folgt 33 Usern
user78174 folgt 31 Usern
user20319 folgt 31 Usern
user4283 folgt 31 Usern
user11039 folgt 31 Usern
user54856 folgt 31 Usern
user34702 folgt 30 Usern
user29803 folgt 30 Usern
user48178 folgt 30 Usern
user21179 folgt 30 Usern
user1733 folgt 30 Usern
user

# Alle Posts eines zufälligen Users

In [22]:
random_user = random.choice(top100)

result = conn.query(f"MATCH (u:User {{name: '{random_user['b']['name']}'}})-[:POSTED]->(t:Tweet) RETURN t")
for tweet in result:
    continue
    #print(tweet['t']['content'])
    #print("")

# Anzahl der Follower eines zufälligen Users

In [23]:
result = conn.query(f"MATCH (follower:User)-[:FOLLOWS]->(user:User {{name: '{random_user['b']['name']}'}}) RETURN count(follower) AS followerCount")
print(result)

[<Record followerCount=749>]


# Anzahl der verfolgten Accounts eines zufälligen Users

In [24]:
result = conn.query(f"MATCH (user:User {{name: '{random_user['b']['name']}'}})-[:FOLLOWS]->(followed:User) RETURN count(followed) AS followedCount")
print(result)
print(random_user['b']['userId'])

[<Record followedCount=105>]
10228272


# Die 25 meistgeliketen Posts der gefolgten Accounts eines zufälligen Users

In [25]:
result = conn.query(f"""
MATCH (follower:User)-[:FOLLOWS]->(followed:User)-[:POSTED]->(tweet:Tweet)
WHERE follower.name = '{random_user['b']['name']}'
WITH follower, collect(followed.name) AS followedUsers
MATCH (t:Tweet)<-[:POSTED]-(u:User)
WHERE u.name IN followedUsers
OPTIONAL MATCH (t)<-[:LIKES]-(liker:User)
RETURN t.content AS content, COUNT(liker) AS numLikes
ORDER BY numLikes DESC
LIMIT 25
""")

for tweet in result:
    print(tweet['content'])
    print("Likes: ", tweet['numLikes'])
    print()

New Roots album avail now!! 
https://t.co/rJdXrSyYr5… http://t.co/fYjdt2dxN8
Likes:  3668

@tariqterry bb I travel
Likes:  3507

Eu tento responder a quantas posso. @babihsd @Cristiano perdi a sessão de perguntas e respostas? Não creio! Faz de novo!
Likes:  3501

We’ll have the Santa Roll with a side of Miso. https://t.co/npP7r4j1Ms https://t.co/MQlhBx32WH
Likes:  3474

.@skyferreira's setting the world record for use of lazers on our show tonight. Tune in. #FallonTonight
Likes:  3447

💭 http://t.co/9Q6v0Znn4N
Likes:  3399

I saved a piece of birthday cake my fans. You made my 20s worth it all.  To spreading love, above all things. ❤️🌎 https://t.co/69Z2JX4ylt
Likes:  3387

So interesting. @BillGates is on our program this evening. Fun. #GatesLetterDOTCOM
Likes:  3132

Let me hear u #ScreamingAndShouting with tweets if u downloaded the song on iTunes... feeling it people?? http://t.co/kaRLNR2W
Likes:  3058

Lady Gaga and @ladystarlightny raid the floors of #affleckspalace  We've been a 

# Die 25 neuesten Posts der gefolgten Accounts eines zufälligen Users

In [26]:
result = conn.query(f"""
MATCH (follower:User)-[:FOLLOWS]->(followed:User)-[:POSTED]->(tweet:Tweet)
WHERE follower.name = '{random_user['b']['name']}'
WITH follower, collect(followed.name) AS followedUsers
MATCH (t:Tweet)<-[:POSTED]-(u:User)
WHERE u.name IN followedUsers
RETURN t.content AS content, t.date_time AS date
ORDER BY t.date_time DESC
LIMIT 25
""")

for tweet in result:
    print(tweet['content'])
    print("Tweeted on: ", tweet['date'])
    print()

“To sit back and let fate play its hand out and never influence it is not the way man was meant to operate.” https://t.co/U7cUApqcz7
Tweeted on:  31/12/2016 23:30

"Don’t count the days; make the days count." https://t.co/uiedUxLbg4
Tweeted on:  31/12/2016 22:37

Gracias a todos por regalarme un año más disfrutando de hacer música y compartiéndola con ustedes! Mucha paz y aleg… https://t.co/B7xJ7YNs7m
Tweeted on:  31/12/2016 22:26

"Despite everything, no one can dictate who you are to other people." https://t.co/Vw9cZsFGqI
Tweeted on:  31/12/2016 21:32

peace 2016 
and thank you ✌🏼️
☄️🙏🏼♡ 
we're gonna be alright 💫💡
♡ 🐇 https://t.co/WtuXfwutzU
Tweeted on:  31/12/2016 20:38

In 2016, #ThisHappened https://t.co/6pZjo8fylG
Tweeted on:  31/12/2016 18:26

In 2016, #ThisHappened https://t.co/6pZjo8fylG
Tweeted on:  31/12/2016 18:26

From Rio to Euro and everything in between. These were the most talked about trends around the globe in 2016.
https://t.co/LQJNkKE8of
Tweeted on:  31/12/2016 17:

# Caching / Fan Out hier

In [27]:
# TODO

# Die 25 beliebtesten Tweets, die die Wörter "hello" und "from" enthalten

In [28]:
result = conn.query("""
MATCH (t:Tweet)
WHERE ALL(word IN [' hello ', ' from '] WHERE t.content CONTAINS word)
WITH t
MATCH (t)<-[:LIKES]-(liker:User)
RETURN t.content AS content, COUNT(liker) AS numLikes
ORDER BY numLikes DESC
LIMIT 25
""")

for tweet in result:
    print(tweet['content'])
    print("Likes: ", tweet['numLikes'])
    print()

Why did @Adele cross the road? She wanted to say hello from the other side.
Likes:  1022

You asked: 
Hello from Japan!
I'm Anna!I I'm a big fan! Please say me hello Anna!
 http://t.co/5Oyi7SiUhY
Likes:  394

You asked: Can you say hello to me ,please? I am from Bulgaria!!! http://t.co/dyNj55sAq4
Likes:  337

You asked: Can you say hello to me?  My name is Narin and I am from Germany http://t.co/lx2CRb3xwh
Likes:  214

#JoanneVibes say hello from backstage at...??? https://t.co/m9PI2oUUo8
Likes:  120

Say hello to Himanshu Yadav and all my friends from #India. I will come shortly with more news. http://t.co/60xEW67c
Likes:  98

You asked: Hi Ronaldo! I love you so much! Please say hello to me! I am from Turkey!  http://t.co/xpDH5Ppb67
Likes:  87

You asked: hello from France, champion ! :) im so proud of you http://t.co/AhDl5iMGzF
Likes:  83

You asked: hello from morocco 
 http://t.co/gbuEyP5953
Likes:  79

